In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm
import pandas as pd
from data_sources import process_parallel, transform_csv_row

In [4]:
from collections import Counter

In [5]:
import scipy.sparse

## Encode music21 stream to text representation 

This notebook uses a full component format 
- measure separators, instruments, and separated octaves
- Format: note, octave, action type, instrument
- note repr: nG# o4 t1 i0

### Load midi data

In [6]:
version = 'v8'
data_path = Path('data/midi')
version_path = data_path/version

In [7]:
source_dir = 'midi_transform'
out_dir = 'midi_npz'
source_csv = version_path/source_dir/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'{out_dir}.csv'

In [8]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,4,15,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,inferred_offset,song_url,instruments,ht_mode,midi_title,title,seconds,midi,inferred_key,quarter_length,...,ht_bpm,artist,ht_key,ht_time_signature,bpm,section,parts,genres,mxl,midi_transform
0,0.0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,1.0,yu-gi-oh,yu-gi-oh-theme-song,25.411765,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,36.0,...,85.0,wayne-sharpe,C,3.0,85.0,intro,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...
1,0.0,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",1.0,yu-gi-oh3,yu-gi-oh-theme-song,15.000000,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,32.0,...,128.0,wayne-sharpe,C,4.0,128.0,chorus,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...
2,5.0,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",1.0,kiefer,kiefer,10.000000,midi_sources/hooktheory/pianoroll/w/what-a-day...,E minor,16.0,...,96.0,what-a-day,C,4.0,96.0,chorus,chorus,Jazz,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...
3,NaN,https://www.hooktheory.com/theorytab/view/weez...,NaN,1.0,My New Song,beverly-hills,NaN,midi_sources/hooktheory/pianoroll/w/weezer/bev...,NaN,NaN,...,128.0,weezer,C,4.0,NaN,intro-and-verse,intro-and-verse,NaN,NaN,NaN
4,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Weezer - Fall Together,fall-together-,10.322581,midi_sources/hooktheory/pianoroll/w/weezer/fal...,A minor,16.0,...,93.0,weezer,C,4.0,93.0,chorus,chorus,Rock,NaN,midi_transform/hooktheory/pianoroll/w/weezer/f...


In [9]:
df_filtered = df.loc[df[source_dir].notna()]; df_filtered.shape

(163266, 23)

In [10]:
def transform_func(file, out_file, row):
    stream = file2stream(file) # 1.
    chordarr = stream2chordarr(stream, max_dur=128) # 2. max_dur = quarter_len * sample_freq (4). 128 = 8 bars
    if row.source != 'hooktheory': # keep hooktheory
        chord_short = compress_chordarr(chordarr)
        delta_trim = chordarr.shape[0] - chord_short.shape[0]
        if delta_trim > 100: print(f'Removed {delta_trim} rests from {file}')
        chordarr = chord_short
    save_chordarr(out_file, chordarr)

In [11]:
from functools import partial
parallel_func = partial(transform_csv_row, 
        transform_func=transform_func,
        base_path=version_path,
        source_dir=source_dir,
        out_dir=out_dir,
        out_extension='.npz'
       )

In [12]:
# for r in df_filtered.iterrows():
#     parallel_func(r)
#     break

In [ ]:
transcribed_files = process_parallel(parallel_func, df_filtered.iterrows(), total=df_filtered.shape[0])

Compressing rests: 56 -> 32
Compressing rests: 192 -> 32
Removed 176 rests from data/midi/v8/midi_transform/freemidi/genre-dance-eletric/Madonna - Beautiful Stranger.mid
Compressing rests: 48 -> 32
Compressing rests: 64 -> 32
Compressing rests: 52 -> 32
Compressing rests: 58 -> 34
Compressing rests: 42 -> 34
Compressing rests: 64 -> 32
Removed 132 rests from data/midi/v8/midi_transform/freemidi/genre-dance-eletric/Fatboy Slim - Praise You.mid
Removed 224 rests from data/midi/v8/midi_transform/freemidi/genre-dance-eletric/Madonna - Frozen.mid
Compressing rests: 81 -> 33
Removed 112 rests from data/midi/v8/midi_transform/freemidi/genre-dance-eletric/Pitbull - I Know You Want Me.mid
Compressing rests: 47 -> 35
Compressing rests: 55 -> 35
Compressing rests: 42 -> 34
Compressing rests: 44 -> 32
Compressing rests: 95 -> 35
Compressing rests: 64 -> 32
Compressing rests: 124 -> 32
Removed 220 rests from data/midi/v8/midi_transform/freemidi/genre-dance-eletric/New Order - Blue Monday.mid
Compre

Compressing rests: 111 -> 35
Compressing rests: 102 -> 34
Compressing rests: 102 -> 34
Compressing rests: 91 -> 35
Compressing rests: 120 -> 32
Compressing rests: 90 -> 34
Compressing rests: 96 -> 32
Compressing rests: 111 -> 35
Compressing rests: 86 -> 34
Compressing rests: 126 -> 34
Compressing rests: 86 -> 34
Compressing rests: 90 -> 34
Compressing rests: 76 -> 32
Compressing rests: 125 -> 33
Compressing rests: 80 -> 32
Compressing rests: 128 -> 32
Compressing rests: 85 -> 33
Compressing rests: 74 -> 34
Compressing rests: 130 -> 34
Compressing rests: 83 -> 35
Compressing rests: 82 -> 34
Compressing rests: 72 -> 32
Compressing rests: 134 -> 34
Compressing rests: 126 -> 34
Compressing rests: 86 -> 34
Compressing rests: 87 -> 35
Compressing rests: 46 -> 34
Compressing rests: 138 -> 34
Compressing rests: 58 -> 34
Compressing rests: 64 -> 32
Compressing rests: 144 -> 32
Compressing rests: 144 -> 32
Compressing rests: 45 -> 33
Compressing rests: 37 -> 33
Compressing rests: 43 -> 35
Compre

Compressing rests: 189 -> 33
Compressing rests: 189 -> 33
Compressing rests: 191 -> 35
Compressing rests: 311 -> 35
Compressing rests: 183 -> 35
Compressing rests: 183 -> 35
Compressing rests: 183 -> 35
Compressing rests: 185 -> 33
Compressing rests: 305 -> 33
Compressing rests: 177 -> 33
Compressing rests: 177 -> 33
Compressing rests: 177 -> 33
Compressing rests: 179 -> 35
Compressing rests: 299 -> 35
Compressing rests: 40 -> 32
Compressing rests: 171 -> 35
Compressing rests: 38 -> 34
Compressing rests: 213 -> 33
Compressing rests: 211 -> 35
Compressing rests: 217 -> 33
Compressing rests: 341 -> 33
Compressing rests: 211 -> 35
Compressing rests: 213 -> 33
Compressing rests: 211 -> 35
Compressing rests: 217 -> 33
Compressing rests: 128 -> 32
Compressing rests: 341 -> 33
Compressing rests: 64 -> 32
Compressing rests: 211 -> 35
Removed 128 rests from data/midi/v8/midi_transform/freemidi/genre-dance-eletric/Four Tops - Walk Away Renee.mid
Compressing rests: 213 -> 33
Compressing rests: 21

Compressing rests: 64 -> 32
Compressing rests: 81 -> 33
Compressing rests: 64 -> 32
Removed 208 rests from data/midi/v8/midi_transform/freemidi/genre-dance-eletric/New Order - Confusion.mid
Error converting midi to sequence cannot reshape array of size 0 into shape (0,newaxis) data/midi/v8/midi_transform/freemidi/genre-dance-eletric/Daft Punk - Aerodynamic.mid
Compressing rests: 58 -> 34
Compressing rests: 60 -> 32
Compressing rests: 128 -> 32
Removed 124 rests from data/midi/v8/midi_transform/freemidi/genre-dance-eletric/Breathe Carolina - No Vacancy.mid
Compressing rests: 113 -> 33
Compressing rests: 63 -> 35
Removed 108 rests from data/midi/v8/midi_transform/freemidi/genre-dance-eletric/Vengaboys - Boom Boom Boom.mid
Compressing rests: 58 -> 34
Compressing rests: 248 -> 32
Compressing rests: 252 -> 32
Removed 548 rests from data/midi/v8/midi_transform/freemidi/genre-dance-eletric/Fatboy Slim - Rockafella Skank.mid
Compressing rests: 49 -> 33
Compressing rests: 65 -> 33
Removed 128 r

Removed 256 rests from data/midi/v8/midi_transform/freemidi/genre-pop/Box Car Racer - There Is (2).mid
Removed 1712 rests from data/midi/v8/midi_transform/freemidi/genre-pop/Backstreet Boys - Spanish Eyes.mid
Compressing rests: 92 -> 32
Removed 108 rests from data/midi/v8/midi_transform/freemidi/genre-pop/David Bowie - Dancing In The Street.mid
Removed 304 rests from data/midi/v8/midi_transform/freemidi/genre-pop/Anita Harris - Playground.mid
Removed 112 rests from data/midi/v8/midi_transform/freemidi/genre-pop/Scorpions - Cant Get Enough.mid
Compressing rests: 192 -> 32
Removed 176 rests from data/midi/v8/midi_transform/freemidi/genre-pop/Madonna - Beautiful Stranger.mid
Compressing rests: 83 -> 35
Removed 112 rests from data/midi/v8/midi_transform/freemidi/genre-pop/Britney Spears - till the world ends.mid
Compressing rests: 288 -> 32
Removed 256 rests from data/midi/v8/midi_transform/freemidi/genre-pop/Idina Menzel - Let it Go 0.2.mid
Compressing rests: 48 -> 32
Compressing rests: 1

...

Compressing rests: 62 -> 34
Compressing rests: 65 -> 33
Removed 124 rests from data/midi/v8/midi_transform/130k_reddit/E/E/E_SWITHE.MID
Compressing rests: 139 -> 35
Removed 152 rests from data/midi/v8/midi_transform/130k_reddit/E/E/En_Vogue_-_Never_Gonna_Get_It.mid
Compressing rests: 111 -> 35
Removed 124 rests from data/midi/v8/midi_transform/130k_reddit/E/E/EAV-Medley-(Medley).mid
Compressing rests: 46 -> 34
Compressing rests: 82 -> 34
Compressing rests: 38 -> 34
Compressing rests: 60 -> 32
Compressing rests: 160 -> 32
Compressing rests: 122 -> 34
Compressing rests: 40 -> 32
Removed 604 rests from data/midi/v8/midi_transform/130k_reddit/E/E/EngClassic01.mid
Compressing rests: 48 -> 32
Removed 112 rests from data/midi/v8/midi_transform/130k_reddit/E/E/eattherich.mid
Removed 144 rests from data/midi/v8/midi_transform/130k_reddit/E/E/ERASURE.I love saturday.mid
Removed 576 rests from data/midi/v8/midi_transform/130k_reddit/E/E/E_CAPRIP.MID
Compressing rests: 2430 -> 34
Removed 2412 rest

Removed 356 rests from data/midi/v8/midi_transform/130k_reddit/O/O/Opening-(From-'The-West-Side-Story').mid
Compressing rests: 47 -> 35
Compressing rests: 47 -> 35
Compressing rests: 41 -> 33
Compressing rests: 44 -> 32
Compressing rests: 92 -> 32
Compressing rests: 48 -> 32
Removed 560 rests from data/midi/v8/midi_transform/130k_reddit/E/E/EAST SIDE.I didn't know.MID
Compressing rests: 65 -> 33
Removed 128 rests from data/midi/v8/midi_transform/130k_reddit/O/O/Our_Lady_Peace_-_Clumsy.mid
Removed 624 rests from data/midi/v8/midi_transform/130k_reddit/O/O/OPUS-ONE.MID
Compressing rests: 128 -> 32
Compressing rests: 52 -> 32
Compressing rests: 56 -> 32
Compressing rests: 92 -> 32
Compressing rests: 55 -> 35
Removed 504 rests from data/midi/v8/midi_transform/130k_reddit/O/O/OneVision2.mid
Compressing rests: 53 -> 33
Compressing rests: 78 -> 34
Compressing rests: 223 -> 35
Removed 616 rests from data/midi/v8/midi_transform/130k_reddit/O/O/Otis_Redding_-_Sitting_on_the_Dock_of_the.mid
Remov

Compressing rests: 65 -> 33
Compressing rests: 49 -> 33
Removed 512 rests from data/midi/v8/midi_transform/130k_reddit/P/P/Pick-Yourself-Up.mid
Removed 144 rests from data/midi/v8/midi_transform/130k_reddit/P/P/PaintItBlack.mid
Compressing rests: 64 -> 32
Compressing rests: 58 -> 34
Compressing rests: 58 -> 34
Compressing rests: 58 -> 34
Compressing rests: 228 -> 32
Compressing rests: 178 -> 34
Removed 428 rests from data/midi/v8/midi_transform/130k_reddit/P/P/Proloog-(From-'The-West-Side-Story').mid
Compressing rests: 46 -> 34
Removed 172 rests from data/midi/v8/midi_transform/130k_reddit/P/P/PulpFiction.mid
Removed 112 rests from data/midi/v8/midi_transform/130k_reddit/P/P/pitbull-we_are_one_ole_ola_feat_jennifer_lopez.mid
Compressing rests: 61 -> 33
Removed 144 rests from data/midi/v8/midi_transform/130k_reddit/P/P/paintitb2.mid
Compressing rests: 56 -> 32
Compressing rests: 59 -> 35
Compressing rests: 51 -> 35
Removed 4720 rests from data/midi/v8/midi_transform/130k_reddit/P/P/Paul

Compressing rests: 128 -> 32
Removed 320 rests from data/midi/v8/midi_transform/130k_reddit/P/P/pomp03.mid
Compressing rests: 388 -> 32
Compressing rests: 48 -> 32
Removed 404 rests from data/midi/v8/midi_transform/130k_reddit/O/O/o_chmsen.mid
Compressing rests: 44 -> 32
Compressing rests: 2087 -> 35
Removed 2228 rests from data/midi/v8/midi_transform/130k_reddit/P/P/Push_-_StrangeWorldversion2__yod_20130128232027.mid
Compressing rests: 48 -> 32
Compressing rests: 64 -> 32
Compressing rests: 59 -> 35
Compressing rests: 61 -> 33
Compressing rests: 96 -> 32
Removed 212 rests from data/midi/v8/midi_transform/130k_reddit/P/P/Pressure-Cooker.mid
Compressing rests: 58 -> 34
Compressing rests: 88 -> 32
Compressing rests: 44 -> 32
Compressing rests: 80 -> 32
Compressing rests: 44 -> 32
Removed 184 rests from data/midi/v8/midi_transform/130k_reddit/P/P/P.MC CARTNEY.Gotta sing, gotta dance K.mid
Removed 272 rests from data/midi/v8/midi_transform/130k_reddit/P/P/Piano-Man-2.mid
Compressing rests:

Compressing rests: 136 -> 32
Removed 28840 rests from data/midi/v8/midi_transform/130k_reddit/N/N/NEWALKAL.MID
Removed 112 rests from data/midi/v8/midi_transform/130k_reddit/N/N/never_ever.mid
Compressing rests: 106 -> 34
Compressing rests: 54 -> 34
Compressing rests: 107 -> 35
Compressing rests: 159 -> 35
Compressing rests: 106 -> 34
Removed 392 rests from data/midi/v8/midi_transform/130k_reddit/N/N/nolimits.mid
Compressing rests: 253 -> 33
Removed 252 rests from data/midi/v8/midi_transform/130k_reddit/P/P/Peter-Allen-Medley.mid
Compressing rests: 67 -> 35
Compressing rests: 131 -> 35
Compressing rests: 67 -> 35
Removed 176 rests from data/midi/v8/midi_transform/130k_reddit/N/N/nexus.mid
Compressing rests: 66 -> 34
Compressing rests: 64 -> 32
Compressing rests: 44 -> 32
Compressing rests: 64 -> 32
Compressing rests: 96 -> 32
Removed 112 rests from data/midi/v8/midi_transform/130k_reddit/N/N/navy2.mid
Compressing rests: 326 -> 34
Removed 292 rests from data/midi/v8/midi_transform/130k_

In [ ]:
tdf = pd.DataFrame(data={out_dir: list(transcribed_files.values())}, index=list(transcribed_files.keys()))

In [ ]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

In [ ]:
merged_df.to_csv(out_csv, index=False); merged_df.head()

### Calculate timesteps

In [ ]:
merged_df = pd.read_csv(out_csv)

In [ ]:
def calc_timesteps(idxrow):
    idx,row = idxrow
    np_file = row[out_dir]
    if not isinstance(np_file, str) or not (Path(version_path)/np_file).exists(): return idx,None
    try:
#         timesteps = scipy.sparse.load_npz(np_file).shape[0]
        timesteps = load_chordarr(Path(version_path)/np_file).shape[0]
        return idx,timesteps
    except Exception as e:
        print('Error reading text', e, np_file)
    return idx, None

In [ ]:
# for r in merged_df.iterrows():
#     calc_timesteps(r)

In [ ]:
file2steps = process_parallel(calc_timesteps, merged_df.iterrows(), total=merged_df.shape[0])

In [ ]:
len_df = pd.DataFrame(data={f'{out_dir}_timesteps': list(file2steps.values())}, index=list(file2steps.keys()))
merged_len_df = merged_df.join(len_df, how='outer');
len_df.shape, merged_df.shape, merged_len_df.shape

In [ ]:
merged_len_df.to_csv(out_csv, index=False); merged_len_df.head()